In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import os.path
import time

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

#from tensorflow.models.image.cifar10 import cifar10
import cifar10
import cifar10_mod

import sys
sys.path.append('/Users/xiejunyi/Dropbox/Research/MultiLevelOptNN/util')
sys.dont_write_bytecode = True

#from extract_weights import extract_weights
from Net2Net import Net2Net_class
from extract_weights import extract_weights
from train_load_net import train_load_net

In [2]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('train_dir', '/Users/xiejunyi/Dropbox/Research/MultiLevelOptNN/checkpoints/cifar10_train',
                           """Directory where to write event logs """
                           """and checkpoint.""")
tf.app.flags.DEFINE_string('train_dir_new', '/Users/xiejunyi/Dropbox/Research/MultiLevelOptNN/checkpoints/cifar10_train_new',
                           """Directory where to write event logs """
                           """and checkpoint.""")
tf.app.flags.DEFINE_integer('max_steps', 1000000,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_boolean('log_device_placement', False,
                            """Whether to log device placement.""")

In [3]:
# download cifar10 dataset
cifar10.maybe_download_and_extract()

In [4]:
graph1 = tf.Graph()

with graph1.as_default():
  global_step = tf.Variable(0, trainable=False, name='global_step')

  # Get images and labels for CIFAR-10.
  images, labels = cifar10_mod.distorted_inputs()

  # Build a Graph that computes the logits predictions from the
  # inference model.
  logits = cifar10_mod.inference(images)

  # Calculate loss.
  loss = cifar10_mod.loss(logits, labels)

  # Build a Graph that trains the model with one batch of examples and
  # updates the model parameters.
  train_op = cifar10_mod.train(loss, global_step)
  
  # saver
  saver = tf.train.Saver(tf.all_variables())
  
  # summary
  summary_op = tf.merge_all_summaries()

  params = extract_weights(FLAGS.train_dir_new, saver)

with tf.Session(graph=graph1) as sess:
    a = sess.graph.get_operation_by_name('train').outputs
    b = sess.graph.get_operation_by_name('total_loss').outputs
    print(a)
    print(b)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
1. Checkpoint path found.

Load checkpoint state:
model_checkpoint_path: "/Users/xiejunyi/Dropbox/Research/MultiLevelOptNN/checkpoints/cifar10_train_new/model.ckpt-0"
all_model_checkpoint_paths: "/Users/xiejunyi/Dropbox/Research/MultiLevelOptNN/checkpoints/cifar10_train_new/model.ckpt-0"

2. Checkpoint restored.

3. Extracted weights info:
global_step:0
()
conv1/weights:0
(5, 5, 3, 64)
conv1/biases:0
(64,)
conv2/weights:0
(5, 5, 64, 64)
conv2/biases:0
(64,)
local3/weights:0
(2304, 768)
local3/biases:0
(768,)
local4/weights:0
(768, 192)
local4/biases:0
(192,)
softmax_linear/weights:0
(192, 10)
softmax_linear/biases:0
(10,)
conv1/weight_loss/avg:0
()
conv2/weight_loss/avg:0
()
local3/weight_loss/avg:0
()
local4/weight_loss/avg:0
()
softmax_linear/weight_loss/avg:0
()
cross_entropy/avg:0
()
total_loss/avg:0
()
conv1/weights/ExponentialMovingAverage:0
(5, 5, 3, 64)
conv1/biases/ExponentialMovingAv

In [ ]:
if tf.gfile.Exists(FLAGS.train_dir):
  key_in = raw_input('Clear previously saved records? (y/n)\n')
  if (key_in == 'y'):
    print("Deleting records...\n")
    tf.gfile.DeleteRecursively(FLAGS.train_dir)
    tf.gfile.MakeDirs(FLAGS.train_dir)

  # extract weights
  key_in = raw_input('Train old or new network? (o/n)\n')
  if (key_in == 'o'):
    print('====================================================')
    print('Train old network')
    print('====================================================')
    train()
  elif (key_in == 'n'):
    print('====================================================')
    print('Train new network')
    print('====================================================')
    print('======================extract weights======================')
    
    print('======================train on new network======================')
    
    train_load_net(graph1, params, FLAGS.train_dir_new,
                   FLAGS.log_device_placement, FLAGS.max_steps, FLAGS.batch_size)